In [21]:
import yfinance as yf
import pandas as pd

symbols = ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'Meta', 'TSLA', 'NVDA', 'ASML', 'ADBE', 'INTC']
data = yf.download(symbols, start='2024-01-01', end='2025-02-18', auto_adjust=False)['Adj Close']
# 計算估值因子（以P/E Ratio為例，這裡我們用股價近似代替）
valuation_factor = data.tail(1).T  # 最新一天的價格，作為估值的代表

# 計算動量因子（過去6個月的回報率）
momentum_factor = data.pct_change(periods=126).tail(1).T  # 126天約為6個月


[*********************100%***********************]  10 of 10 completed


In [22]:
# 因子標準化
# 計算估值因子（以P/E Ratio為例，這裡我們用股價近似代替）
valuation_factor = data.tail(1).T  # 最新一天的價格，作為估值的代表

# 計算動量因子（過去6個月的回報率）
momentum_factor = data.pct_change(periods=126).tail(1).T  # 126天約為6個月

# 因子標準化
valuation_factor = (valuation_factor - valuation_factor.mean()) / valuation_factor.std()
momentum_factor = (momentum_factor - momentum_factor.mean()) / momentum_factor.std()

# 綜合因子得分（權重可自行調整，這裡我們取相等權重）
factor_scores = 0.5 * valuation_factor + 0.5 * momentum_factor

# 將 DataFrame 轉換為 Series，並以 index（股票符號）作為索引
factor_scores = factor_scores.squeeze()  # 將單列 DataFrame 轉換為 Series

# 根據綜合得分進行排序
factor_scores = factor_scores.sort_values(ascending=False)

In [23]:
# 選擇得分最高的2只股票
top_stocks = factor_scores.index[:2]
print("選擇的投資組合:", top_stocks)
print("綜合因子得分:", factor_scores)

選擇的投資組合: Index(['META', 'TSLA'], dtype='object', name='Ticker')
綜合因子得分: Ticker
META     1.187049
TSLA     1.069964
ASML     0.224979
AMZN     0.037571
MSFT    -0.250090
AAPL    -0.364922
ADBE    -0.382413
GOOGL   -0.391949
NVDA    -0.454293
INTC    -0.675896
Name: 2025-02-14 00:00:00, dtype: float64


In [38]:
import backtrader as bt
import backtrader.analyzers as btanalyzers

class FactorInvestmentStrategy(bt.Strategy):
    def __init__(self):
        self.size = 0  # 用於保存持倉量
        self.top_stocks = top_stocks  # 使用之前選定的投資組合

    def next(self):
        if not self.position:
            cash = self.broker.getcash()
            per_stock_cash = cash / len(self.top_stocks)
            for data in self.datas:
                if data._name in self.top_stocks:
                    size = 0  # Initialize size
                    if not pd.isna(data.close[0]):
                        size = int(per_stock_cash / data.close[0])
                    self.buy(data=data, size=size)

cerebro = bt.Cerebro()

# 添加策略
cerebro.addstrategy(FactorInvestmentStrategy)

# 添加選定股票的數據
for symbol in top_stocks:
    data_feed = bt.feeds.PandasData(dataname=yf.download(tickers=symbol, start='2024-01-01', end='2025-02-18', auto_adjust=False)['Adj Close'])
    
    cerebro.adddata(data_feed, name=symbol)

# 設定初始資金
cerebro.broker.setcash(100000.0)

# 添加分析工具
cerebro.addanalyzer(btanalyzers.SharpeRatio, _name="sharpe", timeframe=bt.TimeFrame.Days, annualize=True)
cerebro.addanalyzer(btanalyzers.DrawDown, _name="drawdown")
cerebro.addanalyzer(btanalyzers.Returns, _name="returns")

# 執行回測
result = cerebro.run()
strategy = result[0]

# 提取分析結果
sharpe_ratio = strategy.analyzers.sharpe.get_analysis().get('sharperatio', None)
max_drawdown = strategy.analyzers.drawdown.get_analysis().get('max', {}).get('drawdown', None)
total_return = strategy.analyzers.returns.get_analysis().get('rnorm100', None)

# 打印分析結果
print(f"夏普比率: {sharpe_ratio:.2f}" if sharpe_ratio is not None else "夏普比率: 無法計算")
print(f"最大回撤: {max_drawdown:.2f}%")
print(f"總收益: {total_return:.2f}%")

import matplotlib.pyplot as plt
%matplotlib inline

# 確保數據沒有 NaN 或無限值
for symbol in top_stocks:
    data_feed = yf.download(tickers=symbol, start='2024-01-01', end='2025-02-18', auto_adjust=False)['Adj Close']
    data_feed.dropna(inplace=True)
    data_feed.replace([float('inf'), float('-inf')], float('nan'), inplace=True)
    data_feed.dropna(inplace=True)
    cerebro.adddata(bt.feeds.PandasData(dataname=data_feed), name=symbol)

# 執行回測
cerebro.run()

plt.rcParams['figure.figsize'] = [15, 12]
plt.rcParams.update({'font.size': 12})
img = cerebro.plot(iplot=False)
img[0][0].savefig('backtrader_factor.png')

[*********************100%***********************]  1 of 1 completed


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


夏普比率: nan
最大回撤: 0.00%
總收益: nan%


ValueError: Axis limits cannot be NaN or Inf